In [1]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [9]:
# If the URL doesn't work, visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL to the dataset 
# Note: This is a very large dataset and will take time to download
local_zip = 'D:/Training/kagglecatsanddogs_5340.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('D:/Training/')
zip_ref.close()
print(len(os.listdir('D:/Training/PetImages/Cat/')))
print(len(os.listdir('D:/Training/PetImages/Dog/'))) 
# Expected Output:
# 12501
# 12501

12501
12501


In [10]:
try:
    os.mkdir('D:/Training/cats-v-dogs')
    os.mkdir('D:/Training/cats-v-dogs/training')
    os.mkdir('D:/Training/cats-v-dogs/testing')
    os.mkdir('D:/Training/cats-v-dogs/training/cats')
    os.mkdir('D:/Training/cats-v-dogs/training/dogs')
    os.mkdir('D:/Training/cats-v-dogs/testing/cats')
    os.mkdir('D:/Training/cats-v-dogs/testing/dogs')
except OSError:
    pass

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")
 
    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]
 
    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)
 
    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)
 
 
CAT_SOURCE_DIR = "D:/Training/PetImages/Cat/"
TRAINING_CATS_DIR = "D:/Training/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "D:/Training/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "D:/Training/PetImages/Dog/"
TRAINING_DOGS_DIR = "D:/Training/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "D:/Training/cats-v-dogs/testing/dogs/"
 
split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)
# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [11]:
print(len(os.listdir('D:/Training/cats-v-dogs/training/cats/')))
print(len(os.listdir('D:/Training/cats-v-dogs/training/dogs/')))
print(len(os.listdir('D:/Training/cats-v-dogs/testing/cats/')))
print(len(os.listdir('D:/Training/cats-v-dogs/testing/dogs/')))
# Expected output:
# 11250
# 11250
# 1250
# 1250

11250
11250
1250
1250


In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]) 
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
TRAINING_DIR = "D:/Training/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))
 
VALIDATION_DIR = "D:/Training/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))
 
# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 22499 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [17]:
# Note that this may take some time.
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/15


c:\Users\Sumeed\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 63/225 ━━━━━━━━━━━━━━━━━━━━ 1:34 584ms/step - accuracy: 0.5139 - loss: 0.9152

c:\Users\Sumeed\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


225/225 ━━━━━━━━━━━━━━━━━━━━ 136s 585ms/step - accuracy: 0.5566 - loss: 0.7616 - val_accuracy: 0.7104 - val_loss: 0.5557
Epoch 2/15
225/225 ━━━━━━━━━━━━━━━━━━━━ 55s 241ms/step - accuracy: 0.7056 - loss: 0.5556 - val_accuracy: 0.7520 - val_loss: 0.5024
Epoch 3/15
225/225 ━━━━━━━━━━━━━━━━━━━━ 53s 232ms/step - accuracy: 0.7573 - loss: 0.4885 - val_accuracy: 0.7584 - val_loss: 0.4857
Epoch 4/15
225/225 ━━━━━━━━━━━━━━━━━━━━ 53s 234ms/step - accuracy: 0.7946 - loss: 0.4381 - val_accuracy: 0.7640 - val_loss: 0.4905
Epoch 5/15
225/225 ━━━━━━━━━━━━━━━━━━━━ 54s 239ms/step - accuracy: 0.8222 - loss: 0.3886 - val_accuracy: 0.7972 - val_loss: 0.4307
Epoch 6/15
225/225 ━━━━━━━━━━━━━━━━━━━━ 54s 238ms/step - accuracy: 0.8442 - loss: 0.3500 - val_accuracy: 0.8912 - val_loss: 0.2609
Epoch 7/15
225/225 ━━━━━━━━━━━━━━━━━━━━ 54s 236ms/step - accuracy: 0.8752 - loss: 0.2881 - val_accuracy: 0.9244 - val_loss: 0.1938
Epoch 8/15
225/225 ━━━━━━━━━━━━━━━━━━━━ 53s 233ms/step - accuracy: 0.9035 - loss: 0.2281 - va

In [ ]:
%matplotlib inline
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
 
epochs=range(len(acc)) # Get number of epochs
 
#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()
 
#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()

In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing
import numpy as np
from google.colab import files
from keras.preprocessing import image
 
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")